In [3]:
import pandas as pd
import json

In [4]:
df = pd.read_csv('data/Commercial_NRT Technical Description.csv')
df.shape

(976, 10)

In [5]:
df.head()

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,ref rno - 3,NaN,Updated,NaN


In [6]:
df['L3 Review Status'].unique()

array([nan,
       'In desc source table is mentioned as DimARRService table  and then source is mentioned as static source. \nIs it a static source file? \nRe check and phrase it accordingly.',
       'ref rno - 3', 'Add source table and souce name', 'Ok',
       'Hardcoded 1 or 2 on what condition? For which value it is 1 or 2 add the details.\n\nSurrogateKey generated within UDP NRT to identify unique records of dimkpistate table. \nLogic: When KPIState = "State1" then hardcoded as 1 \nwhen KPIState = "State2" then hardcoded as 2  \n',
       'Hardcoded 1 or 2 on what condition? For which value it is 1 or 2 add the details',
       'Ref rno - 27', 'Add source table name',
       'Instead of ND mention it as Commercial NextDay Datamart within SA&DP / UDPCOmmercial NextDay Datamart',
       'THis is an indirect mapping to DimGeo.\nMention it accordingly and the join logic',
       'In join logic DimGeo is not used', 'Ref rno - 143', 'OK',
       'what is the significance of - referenc

In [7]:
ref_rno_3 = df['L3 Review Status'].iloc[1]

In [8]:
# Filter for rows where values in "L3 - Himanshu: 4-Mar", "L3 Reviewer Comments", "L3 Review Status"
# Are not "Ok" or null

columns_to_check = ["L3 - Himanshu: 4-Mar", "L3 Reviewer Comments", "L3 Review Status"]

# Function to check if a row should be kept
def valid_row(row):
    # Check if at least one column is NOT 'ok' or NaN
    has_meaningful_comment = any([x != 'Ok' and x != 'ok' and pd.notnull(x) for x in row])
    return has_meaningful_comment

# Apply the filtering function
fail_descriptions = df[df[columns_to_check].apply(valid_row, axis=1)]

# Display filtered DataFrame
fail_descriptions

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,ref rno - 3,NaN,Updated,NaN
...,...,...,...,...,...,...,...,...,...,...
931,UDPCommercial_NRT,vwfactsrclosedevents,SRCreatedDateTimeSeconds,Derived attribute of datatype int based on Cre...,"Logic is not clear, check\nupdated\nok",NaN,NaN,NaN,NaN,Updated
936,UDPCommercial_NRT,vwfactsrclosedevents,SRKey,Surrogate key generated within UDP NRT to iden...,ok,Provide the logic for surrogate key generation...,NaN,Ram,NaN,Updated
940,UDPCommercial_NRT,vwfactsrclosedevents,SRModifiedTimeId,Derived by applying decode logic on EventDateT...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated
949,UDPCommercial_NRT,vwfactsrclosedevents,SRScopeTimeDateTimeId,Derived by applying decode logic on StateAnnot...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated


In [9]:
# correct descriptions
# Function to check if a row should be kept
def pass_row(row):
    # Check if at least one column is NOT 'ok' or NaN
    has_meaningful_comment = not any([x != 'Ok' and x != 'ok' and pd.notnull(x) for x in row])
    return has_meaningful_comment

# Apply the filtering function
pass_descriptions = df[df[columns_to_check].apply(pass_row, axis=1)]

# Display filtered DataFrame
pass_descriptions

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
7,UDPCommercial_NRT,vwdimcausetype,CauseType,Direct mapping to CaseType column in Cases tab...,Ok,Ok,NaN,Ram,Updated,NaN
9,UDPCommercial_NRT,vwdimcausetype,CESBICreatedDateTime,Timestamp(UTC) at which record is inserted by ...,Ok,Ok,NaN,Ram,Updated,NaN
10,UDPCommercial_NRT,vwdimcausetype,CESBILastUpdatedDateTime,Timestamp(UTC) at which record is updated by E...,Ok,Ok,NaN,Ram,Updated,NaN
11,UDPCommercial_NRT,vwdimicmsrsnapshot,CreatedDateTime,Direct mapping to CreateDate column in Inciden...,Ok,Ok,NaN,Ram,Updated,NaN
12,UDPCommercial_NRT,vwdimicmsrsnapshot,ModifiedDateTime,Direct mapping to ModifiedDate column coming f...,Ok,Ok,NaN,Ram,Updated,NaN
...,...,...,...,...,...,...,...,...,...,...
971,UDPCommercial_NRT,vwfactsrclosedevents,TransferReason,Direct mapping to TransferReason column in Cas...,ok,NaN,NaN,NaN,NaN,Updated
972,UDPCommercial_NRT,vwfactsrclosedevents,TransferredBy,Direct mapping to UpdatedBy column in Cases t...,ok,NaN,NaN,NaN,NaN,Updated
973,UDPCommercial_NRT,vwfactsrclosedevents,TransferredDateTime,Direct mapping to EventDateTime column in Case...,ok,NaN,NaN,NaN,NaN,Updated
974,UDPCommercial_NRT,vwfactsrclosedevents,TransferredFrom,Direct mapping to TransferredFrom column in Ca...,ok,NaN,NaN,NaN,NaN,Updated


In [10]:
fail_descriptions

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,ref rno - 3,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,ref rno - 3,NaN,Updated,NaN
...,...,...,...,...,...,...,...,...,...,...
931,UDPCommercial_NRT,vwfactsrclosedevents,SRCreatedDateTimeSeconds,Derived attribute of datatype int based on Cre...,"Logic is not clear, check\nupdated\nok",NaN,NaN,NaN,NaN,Updated
936,UDPCommercial_NRT,vwfactsrclosedevents,SRKey,Surrogate key generated within UDP NRT to iden...,ok,Provide the logic for surrogate key generation...,NaN,Ram,NaN,Updated
940,UDPCommercial_NRT,vwfactsrclosedevents,SRModifiedTimeId,Derived by applying decode logic on EventDateT...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated
949,UDPCommercial_NRT,vwfactsrclosedevents,SRScopeTimeDateTimeId,Derived by applying decode logic on StateAnnot...,Logic update by referring to SRCreatedDateTime...,NaN,NaN,NaN,NaN,Updated


In [11]:
# replace all 'ref rno - 3' with ref_rno_3
fail_descriptions['L3 Review Status'] = fail_descriptions['L3 Review Status'].replace('ref rno - 3', ref_rno_3)

/var/folders/58/jr_gdrxd4bn3hwrmrhn8x7mh0000gn/T/ipykernel_50733/1922106112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fail_descriptions['L3 Review Status'] = fail_descriptions['L3 Review Status'].replace('ref rno - 3', ref_rno_3)


In [12]:
fail_descriptions.head()

,FilePath,FileName,FileAttributes,TechnicalDescription,L3 - Himanshu: 4-Mar,L3 Reviewer Comments,L3 Review Status,L3 Reviewer,1st march,4th march
0,UDPCommercial_NRT,vwdimarrservice,ARRServiceId,This Attribute has a static value - <3001> dec...,Ok,Provide the logic for surrogate key generation...,NaN,NaN,Updated,NaN
1,UDPCommercial_NRT,vwdimarrservice,ARRType,Direct mapping to ARRType column coming from s...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
2,UDPCommercial_NRT,vwdimarrservice,ServiceName,Direct mapping to ServiceName column coming fr...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
3,UDPCommercial_NRT,vwdimarrservice,ServicePackageSKU,Direct mapping to ServicePackageSKU column com...,Ok,NaN,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN
4,UDPCommercial_NRT,vwdimarrservicepackage,ARRServicePackageId,"This Attribute has a static value (2001,2002,2...",Ok,Provide the logic for surrogate key generation...,In desc source table is mentioned as DimARRSer...,NaN,Updated,NaN


In [13]:
#output technical description, review columns to json
fail_descriptions[['TechnicalDescription', 'L3 - Himanshu: 4-Mar', 'L3 Reviewer Comments', 'L3 Review Status']].to_json('data/comments.json', orient='records')

In [14]:
# read the JSON file
with open('data/comments.json') as f:
    data = json.load(f)
    